# Section 7

We want to show that valid outcomes of positive support size of at most $5$ have degree $d \leq 7$.

In [1]:
import time
import numpy as np
import chipsplitting.hyperfield.utils as utils
from chipsplitting.hyperfield import (
    HyperfieldPascalForm as HVPascal, 
    HyperfieldHomogeneousLinearSystem as HVLinearSystem,
)
from chipsplitting import pairing_matrix, print_matrix, PascalForm

## Strategy 1: Find all hyperfield solutions up to degree $8 \leq d \leq 41$

We assume $d = 8, ..., 41$. First, we filter all solutions that are not hyperfield roots of all pascal equations. This set is called $S$; it is like a variety generated by all pascal equations in the hyperfield.

Note that our implementation is much faster than the original implementation!

In [2]:
# positive support size
support_size = 5
# hyperfield variety generated by all pascal equations
S = {}
for d in range(8, 42):
    start = time.time()
    base_types = ["diag", "row", "col"]
    A = [HVPascal(d, b, k) for b in base_types for k in range(d + 1)]
    linear_system = HVLinearSystem(A)
    solutions = linear_system.quick_solve_loop(support_size)
    S[d] = solutions
    print(f"d={d}, #solutions={len(solutions)}")
    end = time.time()
    print(f"Elapsed time: {end - start}")

d=8, #solutions=792
Elapsed time: 0.020519256591796875
d=9, #solutions=882
Elapsed time: 0.042946815490722656
d=10, #solutions=950
Elapsed time: 0.07422184944152832
d=11, #solutions=1084
Elapsed time: 0.06981086730957031
d=12, #solutions=1102
Elapsed time: 0.05540919303894043
d=13, #solutions=1212
Elapsed time: 0.06528401374816895
d=14, #solutions=1248
Elapsed time: 0.09876799583435059
d=15, #solutions=1400
Elapsed time: 0.12128686904907227
d=16, #solutions=1400
Elapsed time: 0.17833709716796875
d=17, #solutions=1530
Elapsed time: 0.22858285903930664
d=18, #solutions=1553
Elapsed time: 0.3215038776397705
d=19, #solutions=1723
Elapsed time: 0.36652421951293945
d=20, #solutions=1710
Elapsed time: 0.5260448455810547
d=21, #solutions=1856
Elapsed time: 0.6228179931640625
d=22, #solutions=1863
Elapsed time: 0.7970619201660156
d=23, #solutions=2049
Elapsed time: 0.8991680145263672
d=24, #solutions=2020
Elapsed time: 1.2503411769866943
d=25, #solutions=2182
Elapsed time: 1.332853078842163
d=2

In [3]:
# lets store our result in a json file
# so we don't need to compute the variety again
import json

json_file_name = 'section7_hyperfield_variety.json'
with open(json_file_name, 'w') as json_file:
    json.dump(S, json_file, default=int)

print(f"JSON dump has been written to {json_file_name}")

JSON dump has been written to section7_hyperfield_variety.json


Now, we generated $S$. Every configuration in $S$ can correspond to some valid outcome.

We apply our second filter: the invertibility criterion.

In [4]:
# we load our variety from a json file
import json

json_file_name = 'section7_hyperfield_variety.json'
with open(json_file_name, 'r') as json_file:
    S = json.load(json_file)
    S = {int(key): value for key, value in S.items()}
print("Loaded variety S")
print(S)

Loaded variety S
{8: [[4, 11, 36, 39, 44], [6, 19, 32, 37, 44], [9, 10, 37, 40, 43], [9, 21, 26, 37, 42], [4, 18, 36, 41, 44], [11, 20, 33, 36, 43], [17, 26, 28, 37, 44], [4, 21, 32, 39, 44], [15, 20, 34, 37, 40], [10, 14, 26, 37, 40], [11, 15, 34, 37, 44], [6, 20, 34, 37, 38], [13, 21, 30, 37, 44], [4, 15, 32, 39, 44], [11, 26, 35, 36, 43], [3, 8, 32, 37, 44], [9, 10, 22, 40, 43], [15, 19, 30, 37, 44], [19, 20, 29, 36, 39], [11, 13, 28, 39, 44], [2, 21, 26, 37, 42], [8, 10, 32, 37, 44], [4, 11, 35, 36, 39], [10, 26, 32, 37, 44], [9, 10, 26, 37, 40], [13, 28, 29, 39, 44], [5, 36, 37, 42, 43], [13, 22, 35, 36, 41], [14, 22, 28, 40, 43], [4, 22, 28, 41, 44], [17, 34, 35, 36, 37], [3, 26, 35, 36, 37], [17, 26, 36, 37, 44], [8, 11, 36, 41, 44], [4, 24, 28, 43, 44], [7, 28, 29, 43, 44], [3, 26, 36, 37, 44], [19, 22, 35, 36, 39], [6, 26, 37, 40, 44], [5, 35, 36, 37, 43], [1, 20, 34, 37, 38], [13, 16, 35, 36, 41], [5, 21, 34, 37, 42], [8, 16, 28, 41, 44], [6, 26, 35, 36, 37], [2, 22, 28, 43, 

In [5]:
full_rank = []
for d in range(8,42):
    for config in S[d]:
        # support of negative and positive entries
        support = [(0,0)] + [utils.to_coordinate(c) for c in config]
        basis = list(range(len(support)))
        P = pairing_matrix(d, basis, support)
        rank = np.linalg.matrix_rank(P)

        if rank == len(support):
            full_rank.append(support)
            print(f"Pairing matrix (degree={d}, support={support}, basis={basis}) has rank: {rank}")

Pairing matrix (degree=8, support=[(0, 0), (4, 1), (5, 0), (1, 6), (0, 8), (3, 5)], basis=[0, 1, 2, 3, 4, 5]) has rank: 6
Pairing matrix (degree=8, support=[(0, 0), (3, 0), (0, 4), (5, 1), (1, 7), (4, 4)], basis=[0, 1, 2, 3, 4, 5]) has rank: 6
Pairing matrix (degree=8, support=[(0, 0), (3, 0), (0, 5), (5, 1), (1, 7), (4, 4)], basis=[0, 1, 2, 3, 4, 5]) has rank: 6
Pairing matrix (degree=8, support=[(0, 0), (1, 1), (3, 0), (3, 4), (0, 8), (5, 3)], basis=[0, 1, 2, 3, 4, 5]) has rank: 6
Pairing matrix (degree=8, support=[(0, 0), (3, 0), (0, 6), (5, 1), (1, 7), (4, 4)], basis=[0, 1, 2, 3, 4, 5]) has rank: 6
Pairing matrix (degree=8, support=[(0, 0), (0, 3), (3, 0), (5, 1), (1, 7), (4, 4)], basis=[0, 1, 2, 3, 4, 5]) has rank: 6
Pairing matrix (degree=8, support=[(0, 0), (1, 1), (4, 0), (3, 4), (0, 8), (5, 3)], basis=[0, 1, 2, 3, 4, 5]) has rank: 6
Pairing matrix (degree=8, support=[(0, 0), (3, 1), (5, 0), (1, 6), (0, 8), (3, 5)], basis=[0, 1, 2, 3, 4, 5]) has rank: 6


In [6]:
for support in full_rank:
    P = pairing_matrix(8, [3,4,5,6,7,8], support)
    rank = np.linalg.matrix_rank(P)
    if rank < 6:
        print("Found pairing matrix with rank less than 6")
    else:
        print("Full rank")

Found pairing matrix with rank less than 6
Found pairing matrix with rank less than 6
Found pairing matrix with rank less than 6
Found pairing matrix with rank less than 6
Found pairing matrix with rank less than 6
Found pairing matrix with rank less than 6
Found pairing matrix with rank less than 6
Found pairing matrix with rank less than 6


**Conclusion**: For $8 \leq d \leq 42$ no valid outcome of degree $d$ exist.

## Strategy 2: Find contracted, hyperfield solutions

We know that some pascal equations can be represented in a *contracted* way (using only 64 variables). Hence, a valid outcome of support size 5 must satisfy all these contracted pascal equations.

**Task**: We count all contracted outcomes $w \in \{0, \pm 1\}^{64}$ that are lie in the variety $A$ generated by specific contracted pascal equations.

We assume $d \geq 11$. Define $\Gamma_d$ to be the set of the following pascal equations
```
HVPascal(d, "diag", 1), HVPascal(d, "diag", 2), HVPascal(d, "diag", 3), 
HVPascal(d, "diag", d - 3), HVPascal(d, "diag", d - 2), HVPascal(d, "diag", d - 1),
HVPascal(d, "row", 1), HVPascal(d, "row", 2), HVPascal(d, "row", 3),
HVPascal(d, "col", 1), HVPascal(d, "col", 2), HVPascal(d, "col", 3),
HVPascal(d, "row", d - 3), HVPascal(d, "row", d - 2), HVPascal(d, "row", d - 1), HVPascal(d, "row", d),
HVPascal(d, "col", d - 3), HVPascal(d, "col", d - 2), HVPascal(d, "col", d - 1), HVPascal(d, "col", d).
```
For each of the following pascal equations $p_d \in \Gamma_d$,
the cardinalities of $\{ \mathrm{contraction}(p_d) : d \geq 11, d \text{ is odd} \}$ and $\{ \mathrm{contraction}(p_d) : d \geq 12, d \text{ is even} \}$ is one.

To explicitly describe the generators of $A$, we fix $d=13$ for the odd case and $d= 14$ for the even case (we could also take any other odd $d \geq 11$ and even $d \geq 12$). For each pascal equation $p_d$ we contract it to obtain a contracted pascal form $\mathrm{contracted}(p_d)$. By the previous paragraph, we saw that $\mathrm{contracted}(p_d) = \mathrm{contracted}(p_{d + 2i})$ for all $i \geq 0$.

It is our task now to find $x \in \{ 0, \pm 1 \}^{64}$ that are roots of all $\mathrm{contracted}(p_d)$, $p \in \Gamma_d$, $d \geq 11$.

### Brute force solution

We can't test all $d \geq 11$ since there are infinitely many $d$. However, it suffices to fix $d = 13$ or $d = 14$ and contract these pascal equations. Then, we enumerate all $\{0,...,63\}^5$ possible configurations of *valid* degree (that is at least one diagonal component is non-zero) and check if they are roots of contracted pascal equations.

**Pseudocode**
```
def solve():
    solutions = []
    for config in configurations:
        if config is a root of all p in contracted_pascal_eqns:
            solutions.push(config)
    return solutions
```

**Complexity**: 64 * 63 * 62 * 61 * 60 = 914,941,440 configurations must be checked.

In [7]:
def countValidConfigsForContractions(degree="even"):
    assert degree == "even" or degree == "odd", "degree must be 'even' or 'odd'"

    enum_indices = [ 
        0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 24, 25, 26, 28, 29, 30, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 48, 49, 50,
       51, 52, 53, 54, 55, 57, 58, 59, 61, 62, 63,
    ]
    diag_indices = [19,23,27,31,56,60,47,46,45,44]
    indices = enum_indices + diag_indices
    
    def create_hat_exhaust_generator():
        # for d in range(63, 15, -1):
        for d in range(len(enum_indices), len(indices)):
            for i in range(1, d):
                for j in range(1, i):
                    for k in range(1, j):
                        for l in range(1, k):
                            yield (indices[d], indices[i], indices[j], indices[k], indices[l])
                    
    res = []
    d = 13 if degree == "odd" else 14
    A = list(map(lambda a: a.contract(), [
        HVPascal(d, "diag", 1), HVPascal(d, "diag", 2), HVPascal(d, "diag", 3), 
        HVPascal(d, "diag", d - 3), HVPascal(d, "diag", d - 2), HVPascal(d, "diag", d - 1),
        HVPascal(d, "row", 1), HVPascal(d, "row", 2), HVPascal(d, "row", 3),
        HVPascal(d, "col", 1), HVPascal(d, "col", 2), HVPascal(d, "col", 3),
        HVPascal(d, "row", d - 3), HVPascal(d, "row", d - 2), HVPascal(d, "row", d - 1), HVPascal(d, "row", d),
        HVPascal(d, "col", d - 3), HVPascal(d, "col", d - 2), HVPascal(d, "col", d - 1), HVPascal(d, "col", d),
    ]))

    gen = create_hat_exhaust_generator()

    for supp in gen:
        w = np.array([1 if idx in supp else 0 for idx in range(64)])
        w[0] = -1
    
        is_valid = True
        for f in A:
            val = f(w)
            if not np.isnan(val):
                is_valid = False
                break
        
        if is_valid:
            res.append(w)

    return res

In [8]:
%%time
res1 = countValidConfigsForContractions("even")
print(f"Number of configurations for even d: {len(res1)}")

Number of configurations for even d: 1283
CPU times: user 4min 32s, sys: 66.1 ms, total: 4min 33s
Wall time: 4min 34s


In [9]:
%%time
res2 = countValidConfigsForContractions("odd")
print(f"Number of configurations for even d: {len(res2)}")

Number of configurations for even d: 1265
CPU times: user 4min 34s, sys: 50.4 ms, total: 4min 34s
Wall time: 4min 35s


## Make it faster

**Problem:** Enumerating all possible 5-tuples $[64]^{5}$ is too expensive. 

**Solution:** Each pascal equation $p$ defines constraints $\mathrm{supp}^+(p)$ and $\mathrm{supp}^-(p)$.

We assume that any potential hyperfield solution $x$ has only negative entries for $x_{00}$, i.e. $x_{00} = -1$.

- Case 1: $(0,0) \in \mathrm{supp}^+(p)$. Then set some $x_{ij} = 1$ for some $(i,j) \in \mathrm{supp}^+(p) \setminus \{(0,0) \}$.
- Case 2: $(0,0) \in \mathrm{supp}^-(p)$. Then set some $x_{ij} = 1$ for some $(i,j) \in \mathrm{supp}^-(p) \setminus \{(0,0) \}$.
- Case 3: $\mathrm{supp}^+(p)$ and $\mathrm{supp}^-(p)$ are non-empty. Pick $x_{ij}, x_{kl}$ from $(i,j) \in \mathrm{supp}^+(p) \setminus \{(0,0) \}$ and $(k,l) \in \mathrm{supp}^-(p) \setminus \{(0,0) \}$.
- Otherwise: No solution exists.

**Pseudocode**
```

def quick_solve():
    def make_constraints():
        constraints = []
        for p in contracted_pascal_eqns:
            positive_support, negative_support = p.support
            if 0 in positive_support:
                push [i : i in positive_support, i != 0] to constraints
            else if 0 in negative_support:
                push [i : i in negative_support, i != 0] to constraints
            else if positive_support and negative_support are non empty:
                push [i : i in positive_support] to constraints
                push [i : i in negative_support] to constraints
            else:
                Error: p has no root!
        return constraints

    solutions = {x | for all c in constraints at least one 0 <= j <= 63 exists such that x[j] != 0 and j in c}
    return solutions
```

In [10]:
%%time
d = 13
A = list(map(lambda a: a.contract(), [
    HVPascal(d, "diag", 1), HVPascal(d, "diag", 2), HVPascal(d, "diag", 3), 
    HVPascal(d, "diag", d - 3), HVPascal(d, "diag", d - 2), HVPascal(d, "diag", d - 1),
    HVPascal(d, "row", 1), HVPascal(d, "row", 2), HVPascal(d, "row", 3),
    HVPascal(d, "col", 1), HVPascal(d, "col", 2), HVPascal(d, "col", 3),
    HVPascal(d, "row", d - 3), HVPascal(d, "row", d - 2), HVPascal(d, "row", d - 1), HVPascal(d, "row", d),
    HVPascal(d, "col", d - 3), HVPascal(d, "col", d - 2), HVPascal(d, "col", d - 1), HVPascal(d, "col", d),
]))
linear_system = HVLinearSystem(A)
solutions_odd = linear_system.quick_solve_loop(5)
print(len(solutions_odd))

1265
CPU times: user 14.6 ms, sys: 2.27 ms, total: 16.8 ms
Wall time: 16.1 ms


In [11]:
%%time
d = 14
A = list(map(lambda a: a.contract(), [
    HVPascal(d, "diag", 1), HVPascal(d, "diag", 2), HVPascal(d, "diag", 3), 
    HVPascal(d, "diag", d - 3), HVPascal(d, "diag", d - 2), HVPascal(d, "diag", d - 1),
    HVPascal(d, "row", 1), HVPascal(d, "row", 2), HVPascal(d, "row", 3),
    HVPascal(d, "col", 1), HVPascal(d, "col", 2), HVPascal(d, "col", 3),
    HVPascal(d, "row", d - 3), HVPascal(d, "row", d - 2), HVPascal(d, "row", d - 1), HVPascal(d, "row", d),
    HVPascal(d, "col", d - 3), HVPascal(d, "col", d - 2), HVPascal(d, "col", d - 1), HVPascal(d, "col", d),
]))
linear_system = HVLinearSystem(A)
solutions_even = linear_system.quick_solve_loop(5)
print(len(solutions_even))

1283
CPU times: user 14.7 ms, sys: 1.08 ms, total: 15.8 ms
Wall time: 15 ms


In [12]:
unique_solutions = list(set([*solutions_odd, *solutions_even]))
len(unique_solutions)

2318

We see that $\Gamma^{even} \cup \Gamma^{odd}$ contains 2318 solutions.

## Strategy 3: Contract contracted hyperfield solutions further

### Lemma 7.4

**Contraction step**: The next step is to apply the contraction transformation $\chi$ that maps $d^{(0)}$ and $d^{(1)}$ to their sum $d^{(0)}+d^{(1)}$.

We show that $\Lambda \coloneqq \chi(\Gamma^{even} \cup \Gamma^{odd})$ only contains configurations that have positive support size $5$ except for one case. We call solutions $\omega \in \Lambda$ *super contracted solutions*. 

In [13]:
d0 = set([56, 57, 58, 59])
d1 = set([60, 61, 62, 63])
for config in unique_solutions:
	if set(config).intersection(d0) and set(config).intersection(d1):
		print(f"Config {config} has positive support 4")

Config (3, 5, 12, 56, 60) has positive support 4


We see that only for one $\theta$ the image $\chi(\theta)$ has positive support size smaller than $5$. The $\theta$ is of the form $(3, 5, 12, 56, 60)$ which corresponds to the hyperfield configuration with positive support $x_{0,3}, x_{1,1}, x_{3,0}, d_{0}^{(0)}, d_0^{(1)}$.

In the next cell we see that $\Lambda = \chi(\Gamma^{even} \cup \Gamma^{odd})$ contains exactly 2290 configurations.

In [14]:
def chi(config):
    t = [x for x in config]
    for x in [60, 61, 62, 63]:
        if x in t:
            t.remove(x)
            t.append(x - 4)
    t.sort()
    return tuple(set(t))    

In [15]:
# count the number of unique elements in Lambda
Lambda = set()
for config in unique_solutions:
    Lambda.add(chi(config))
len(Lambda)

2290

Of these 2290 elements one element has only positive support size $4$. Thus, there are only 2289 elements of positive support size $5$.

### Exclude config of positive support size 4

Let's take a closer look at the hyperfield config $\omega$ with positive support $x_{0,3}, x_{1,1}, x_{3,0}, d_{0}^{(0)}, d_0^{(1)}$. There are infinitely many chipsplitting configurations $w \in \mathbb Z^{V_d}$ that map to this hyperfield config. Using the invertibility criterion we can exclude all of these chipsplitting configurations from being valid outcomes.

For any $d$ we set $\ell = d + 1$ and $\lambda = (1, ..., 1) \in \mathbb Z^{\ell}$. We see that
- $S_1 = {(0,3)}, E_1 = {0}$ or $S_1 = E_1 = \emptyset$
- $S_2 = {(1,1)}, E_2 = {1}$ or $S_2 = E_2 = \emptyset$
- $S_4 = {(3,0)}, E_4 = {3}$ or $S_4 = E_4 = \emptyset$
- $S_{i+1} = {(i,d-i)}, E_{i+1} = {i}$ for some $i \geq 4$
- $S_{j+1} = {(j,d-i)}, E_{j+1} = {j}$ for some $j\geq 4, i \neq j$
- $S_k = E_k = \emptyset$ for all $k$ that is not $1,2,4,i,j$

These are all possibilities for pairing matrices that occur for outcomes that map to $\omega$. We compute the determinant of all pairing matrices for all $d$, which is possible, and see that it is nonzero. By the invertibility criterion and Proposition 5.4, every outcome with support contained in $S$ must be the initial configuration.

**Conclusion**: It remains to check 2289 cases.

In [16]:
degree = 14 # degree doesnt matter, pairing matrix is always one. see pdf on ipad for simple proof
units = [0]
supports = [(0,3)]
pairing_matrix(degree, units, supports)

[[1]]

In [17]:
degree = 14
units = [1]
supports = [(1,1)]
pairing_matrix(degree, units, supports)

[[1]]

In [18]:
degree = 14
units = [3]
supports = [(3,0)]
pairing_matrix(degree, units, supports)

[[1]]

### So far we have remaining 2289 super contracted hyperfield solutions.

We investigate $\Lambda$ further.

First let us save the set $\Lambda$ for later retrieval.

In [19]:
import json

if not unique_solutions:
    assert False, "unique_solutions is empty"

# count the number of unique elements in Lambda
Lambda = set()
for config in unique_solutions:
    if config == (3, 5, 12, 56, 60):
        continue
    Lambda.add(chi(config))

# lets store our result in a json file
# so we don't need to compute the variety again
json_file_name = 'section7_Lambda.json'
with open(json_file_name, 'w') as json_file:
    json.dump(list(Lambda), json_file, default=int)

print(f"JSON dump has been written to {json_file_name}")

JSON dump has been written to section7_Lambda.json


Load $\Lambda$.

In [20]:
json_file_name = 'section7_Lambda.json'
with open(json_file_name, 'r') as json_file:
    Lambda = [tuple(sorted(x)) for x in json.load(json_file)]
print(f"Loaded JSON Lambda. It contains {len(Lambda)} items.")

Loaded JSON Lambda. It contains 2289 items.



**Lemma 7.6**
Super contracted solutions in $\Lambda$ have at monst one nonzero component $c_0, ..., c_3$.

```
*
**
***
****
.....
......
.......
........
.........
..........
...........
............
**** ccc ****
**** cccc ****
**** ccccc ****
**** cccccc ****
```

In [21]:
c_indexes = { 48, 49, 50, 51 }
r_indexes = { 52, 53, 54, 55 }
d_indexes = { 56, 57, 58, 59 }

for x in Lambda:
    intersections = {
        "c": set(x).intersection(c_indexes),
        "r": set(x).intersection(r_indexes),
        "d": set(x).intersection(d_indexes),
    }

    for key, intersection in intersections.items():
        if len(intersection) not in { 0,1 }:
            print(f"Error: {x} and key {key} invalid")

print("Done. If no error message appeared, then Lemma 7.6 is proved. □")

Done. If no error message appeared, then Lemma 7.6 is proved. □


**Example 7.8**

See my notes for a proof.

In [22]:
def toy_example_78():
    degree = 90
    units = [0,1,2]
    support = [(0,0), (0,degree), (1,3)]
    P = pairing_matrix(degree, units, support)
    rank = np.linalg.matrix_rank(P)
    if rank > 0:
        print(f"Pairing matrix is invertible since it has rank {rank}.")
    else:
        print("Pairing matrix is NOT invertible")
    print_matrix(P)
    # for debugging
    # print(PascalForm(12, "diag", 2))

toy_example_78()

Pairing matrix is invertible since it has rank 3.
   1    1    0 
  90    0    1 
4005    0   86 
